<a href="https://colab.research.google.com/github/AprendizajeProfundo2021/trabajo-equipo7/blob/main/Datasets/Dataset_Detecci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descargando banco de datos

In [ ]:
import numpy as np
import os,shutil

In [ ]:
%%capture
!wget http://www.tamirhassan.com/html/files/icdar2013-competition-dataset-with-gt.zip -O Marmotdataset.zip
!unzip Marmotdataset

In [ ]:
%%capture
!unzip /content/drive/MyDrive/icdar2013.zip

## Procesando primer banco de datos
- Preparado para la detección de celdas

In [ ]:
import xml.etree.ElementTree as ET
import json

In [ ]:
%%capture
!pip install pdf2image
!sudo apt-get install poppler-utils
from pdf2image import convert_from_path

In [ ]:
T=['train','test']
subT=['images','annotations','yolo']

os.mkdir('celldataset') 
for x in T:
  os.mkdir('celldataset/'+x)
  for y in subT: 
    os.mkdir('celldataset/'+x+'/'+y)

In [ ]:
%%capture
pip install PyPDF2

In [ ]:
from PyPDF2 import PdfFileReader

In [ ]:
%%capture
pip install pdfrw

In [ ]:
from pdfrw import PdfReader

In [ ]:
from PIL import Image

In [ ]:
def TranslatePoints(src, pdf, im,pag):
    sx0, sy0, sx1, sy1 = src
    ssx, ssy = (int(pdf.pages[pag].MediaBox[2].split('.')[0]),int(pdf.pages[pag].MediaBox[3].split('.')[0]))
    dsx, dsy = im.size
    sy01 = ssy-sy1
    sy11 = ssy-sy0
    x0 = sx0/ssx*im.size[0]
    x1 = sx1/ssx*im.size[0]
    y0 = sy01/ssy*im.size[1]
    y1 = sy11/ssy*im.size[1]
    return (int(x0), int(y0), int(x1), int(y1))

In [ ]:
def convert_yolo(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
from pathlib import Path
import json

In [ ]:
list_dir=['/content/icdar2013/test/JPEGImages','/content/icdar2013/train/JPEGImages']
k=0
for path in list_dir:
  for file in os.listdir(path):
    tipo=file[:2]
    numero=file[3:file[3:].find('-')+3]
    x=file.find('.')
    pagina=int(file[x-2:x])
    ruta='/content/competition-dataset-'+ tipo+'/'+tipo+'-'+numero

    path_pdf=ruta+'.pdf'
    path_reg=ruta+'-reg.xml'
    path_str=ruta+'-str.xml'

    pdf = PdfReader(path_pdf)
    im = Image.open(path+'/'+file)

    conjunto=Path(path).parent.stem
    path_dataset='/content/celldataset/'+conjunto+'/'

    #Para las coordenadas de la tabla
    coordenadas_tabla=ET.parse(path_reg)
    coordenadas_tabla = coordenadas_tabla.getroot()
    coordenadas_tabla = coordenadas_tabla.findall('table')

    #Para las coordenadas de las celdas
    coordenadas_celdas=ET.parse(path_str)
    coordenadas_celdas = coordenadas_celdas.getroot()
    

    for tabla in coordenadas_tabla:
      anotaciones = []
      pag_tabla=int(tabla.find('region').get('page'))
      if pag_tabla==pagina and tipo+'-'+numero!='eu-015':
        coords = tabla.find('region').find('bounding-box')
        id=tabla.get('id')
        yolo_txt=open(path_dataset+'yolo/'+tipo+'-'+numero+'-'+str(pagina)+'-'+id+'.txt','w')
        x1,y1,x2,y2 = int(coords.get('x1')),int(coords.get('y1')),int(coords.get('x2')),int(coords.get('y2'))
        x1,y1,x2,y2=TranslatePoints((x1,y1,x2,y2),pdf,im,pagina-1)
        x1,y1,x2,y2 = x1-5,y1-5,x2+10,y2+10
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.crop(box=[x1,y1,x2,y2]).save(path_dataset+'images/'+tipo+'-'+numero+'-'+str(pagina)+'-'+id+'.jpg', 'PNG')
        celdas = coordenadas_celdas.findall('table')[int(id)-1].find('region').findall('cell')
        for celda in celdas:
          box = celda.find('bounding-box')
          if box.get('x1') != '26ß':
            xmin,ymin,xmax,ymax = int(box.get('x1')),int(box.get('y1')),int(box.get('x2')),int(box.get('y2'))
          else:
            xmin,ymin,xmax,ymax = 268,int(box.get('y1')),int(box.get('x2')),int(box.get('y2'))
          xmin,ymin,xmax,ymax=TranslatePoints((xmin,ymin,xmax,ymax),pdf,im,pagina-1)
          xmin=xmin-x1
          ymin=ymin-y1
          xmax=xmax-x1
          ymax=ymax-y1
          # anotaciones --> coordenadas referenciadas a la tabla
          anotaciones.append({'bbox':[xmin,ymin,xmax,ymax],'label':'cell'})
          # anotaciones --> coordenadas yolo
          x,y,w,h = convert_yolo((x2-x1,y2-y1), (xmin,xmax,ymin,ymax))
          yolo_txt.write('0 ' + str(x) + ' ' + str(y)+ ' ' + str(w) + ' ' + str(h) + '\n')

        yolo_txt.close()
        txt=open(path_dataset+'annotations/'+tipo+'-'+numero+'-'+str(pagina)+'-'+id+'.txt', 'w')
        with txt as json_file:
          json.dump(anotaciones, json_file)

### Guardamos el dataset en zip para su uso previo

In [ ]:
import shutil
shutil.make_archive('/content/drive/MyDrive/celldataset','zip','/content/celldataset')

'/content/drive/MyDrive/celldataset.zip'